# Trabalho 1 - Regressão Linear Bayesiana
---

A **Regressão Linear Bayesiana**, diferente da regressão linear, visa encontrar uma distribuição de probabilidade que represente os possíveis modelos lineares que representem os dados, ou seja, enquanto a regressão linear busca encontrar um vetor de pesos *w* tal que se aproxime de $y - w^{T}x \approx 0$ , a regressão linear bayesiana busca encontrar $p(w) = \mathcal{N}(w|\mu,\sigma^2)$ que também se aproxime.

As implementações feitas neste notebook têm como base as explicações da seguinte [explicação](http://krasserm.github.io/2019/02/23/bayesian-linear-regression/). Entretanto, para melhor entendimento do conteúdo, todo o código foi reescrito e comentado

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image

In [2]:
# funções auxiliares de plot
def plot_data(x,y, label=True):
    if label:
        plt.scatter(x,y,marker='o',c='k',s=50, label="Dados de treino")
    else:
        plt.scatter(x,y,marker='o',c='k',s=50)
    
def plot_expected(x,y, label=True):
    if label:
        plt.scatter(x,y,marker='x', color='r',label="Valor esperado")
    else: 
        plt.scatter(x,y,marker='x', color='r')

def plot_prediction(x,y,var, std_times=1):
    y = y.ravel() #flatten
    std = np.sqrt(var.ravel()) * std_times #flatten
    
    plt.plot(x,y, label="Predição")
    plt.fill_between(x.ravel(), y+std, y-std, alpha=.5, label="Incerteza")
    
def plot_model(x,y, label=False):
    if label:
        plt.plot(x, y, 'r--', alpha=.5, label="Modelos a posteriori")
    else: 
        plt.plot(x, y, 'r--', alpha=.5)
    
def plot_models(x, ys):
    plot_model(x,ys[:,0], label=True)
    for i in range(1, ys.shape[1]):
        plot_model(x,ys[:,i])
        
def plot_posteriori_distribution(mean, cov, resolution=100):
    x = y = np.linspace(-1,1,resolution)
    
    grid = np.dstack(np.meshgrid(x,y)).reshape(-1,2)
    densities = multivariate_normal.pdf(grid, mean=mean.ravel(), cov=cov).reshape(resolution, resolution)
    
    plt.imshow(densities, origin='lower', extent=(-1,1,-1,1))

No modelo de regressão bayesiana, os dados são representados utilizando funções de base $\Phi$. Para este trabalho, utilizarei duas funções de base:

* Função de base identidade, onde $\Phi(X) = X$ (para a primeira questão);
* Função de base polinomial, onde $\Phi(X) = [x^1, x^2, ..., x^k]^T$ , onde $k$ é um número inteiro (por padrão, 5)

In [3]:
def add_bias_parameter(x):
    '''Adiciona os termos independentes (parâmetro bias) aos dados'''
    return np.concatenate([np.ones(x.shape), x], axis=1)

identity_basis_function = lambda x: x

def polynomial_basis_function(x, degree=5):
    return np.concatenate([x**d for d in range(1,degree+1)], axis=1)

Assim como na regressão linear, vamos assumir valores iniciais para *w* (no caso, uma distribuição inicial). Essa *distribuição a priori* de *w* será, por convenção, uma distribuição com média 0 e com uma precisão $\alpha$, tal que
$$p(w|\alpha) = \mathcal{N}(w|\alpha^{⁻1}I)$$

Após N amostras, com entrada $\Phi$ e saídas ***y***, a distribuição de *w* continua sendo uma normal, onde:

* sua média é $m_N = S_N(S_0^{-1}m_0 + \beta \Phi^{T}y)$. Como a distribuição a priori de *w* tem média 0, a fórmula simplifica para $m_N = \beta S_n\Phi^{T}y$

* a matriz inversa de sua variância é $S_N^{-1} = S_0^{-1} + \beta \Phi^T \Phi$. Como $S_0 = \alpha^{⁻1}I$, $S_0^{-1} = \alpha I$. Portanto, a fórmula da inversa da variância fica $S_N^{-1} = \alpha I + \beta \Phi^T \Phi$

Agora que temos a distribuição a posteriori de w $p(w| x,\textbf{y}, \alpha, \beta) = \mathcal{N}(w|m_N, S_N)$, podemos encontrar as distribuições preditivas $p(t| x,\textbf{y}, \alpha, \beta) = \mathcal{N}(t|m_N^T \Phi(x), \sigma_N^2(x))$, onde $\sigma_N^2(x) = \frac{1}{\beta} + \Phi(x)^T S_N \Phi(x)$

In [4]:
def posteriori(x, y, alpha, beta):
    '''p(w|t) = N(w|mean,var)'''
    var_inv = alpha*np.eye(x.shape[1]) + beta*x.T.dot(x) # var_inv = alpha * I + beta * xT * x
    var = np.linalg.inv(var_inv) # var_inv_inv
    mean = beta * var.dot(x.T).dot(y) # var * ( var_0_inv * mean_0 + beta * xT * y ), onde mean_0 = 0
    
    return mean, var

def predict(x, mean, var, beta):
    '''Como w segue uma distribuição normal, o valor mais provável de w será a esperança da distribuição,
    que, no caso da normal, é a média. Logo:
    y = x * mean
    '''
    y = x.dot(mean)
    var = (1 / beta) + np.sum(x.dot(var) * x, axis = 1)
    
    return y, var

A função auxiliar abaixo condensa o fluxo principal do treinamento:
* é carregado um conjunto de dados de um arquivo;
* é aplicada uma função de base sobre X
* é pega uma amostra de tamanho N dos dados
* o modelo é treinado com N amostras
* são realizados os plots de:
    * densidade da probabilidade a posteriori dos pesos
    * 5 modelos que seguem a distribuição da posteriori, bem como os valores esperados da predição
    * a esperança da posteriori, com uma margem de 2 desvios padrões

In [5]:
def generate_plot_function(file, samples_sizes=[1, 3, 5, 7, 10], basis_function=identity_basis_function, figsize=(20,20), models_title=True):
    data = np.loadtxt(file, delimiter=',')
    data = data[np.argsort(data[:,0])]
    X = data[:,0].reshape(-1,1)
    Y = data[:,1].reshape(-1,1)

    phi = add_bias_parameter(basis_function(X))

    n_experiments = len(samples_sizes)
    
    n_plots = 3
    if phi.shape[1] > 2:
        n_plots = 2
    
    def plot(alpha, beta):
        plt.figure(figsize=figsize)
        plt.subplots_adjust(hspace=.4)

        for i, n in enumerate(samples_sizes):
            # obtendo os elementos da amostra
            x_n = X[:n]
            phi_n = phi[:n] # dados de treino
            y_n = Y[:n] # valores de saída

            # treinando o modelo
            mean, var = posteriori(phi_n, y_n, alpha, beta) # computando a posteriori de w
            y, y_var = predict(phi, mean, var, beta) # prevendo valores para Y, dado os parâmetros da distribuição de w

            # obtendo 5 exemplos de modelo a partir da distribuição de w
            models_samples = np.random.multivariate_normal(mean.ravel(), var, 5).T # como w segue uma distribuição normal, os modelos gerados aleatoriamente seguind oessa distruibuição representam bem os dados
            prediction_samples = phi.dot(models_samples) # realizando a predição de Y utilizando os modelos 

            # plots
            plot_i = 1
            if n_plots == 3:
                plt.subplot(n_experiments, n_plots, i*n_plots + plot_i)
                plot_posteriori_distribution(mean, var) # distribuição posteriori dos pesos a partir das amostras
                plt.title(f"Posteriori com {n} amostras")
                
                plot_i += 1

            plt.subplot(n_experiments, n_plots, i*n_plots + plot_i)
            plot_expected(X, Y) # valores esperados para a predição
            plot_data(x_n, y_n) # valores utilizados para o treinamento do modelo
            plot_models(X, prediction_samples) # valores preditos pelos modelos que seguem a distribuição calculada
            if models_title:
                var_string = str(np.diag(np.round(var, 3)))
                plt.title(f"Médias: {np.round(mean.ravel(), 3)} | Var. : {var_string} | Cov. : {np.round(var[0,1], 3)}")
            else:
                plt.title(f"Modelos gerados analisando {n} amostras")
            plt.legend()   
            
            plot_i += 1

            plt.subplot(n_experiments, n_plots, i*n_plots + plot_i)
            plot_expected(X, Y, label=False) # valores esperados para a predição
            plot_data(x_n, y_n, label=False) # valores utilizados para o treinamento do modelo
            plot_prediction(X, y, np.sqrt(y_var), std_times=2) # valores predito pela esperança da distribuição de w
            plt.title(f"Variância média: {np.round(np.mean(y_var), 3)}")
            plt.legend()
            
    return plot

## Questão 1

Implemente um modelo de regressão linear Bayesiana para os dados disponı́veis em *linear_regression_data.csv*.

* Apresente um gráfico contendo os dados e uma representação da distribuição preditiva encontrada.

* Esta representação consistirá na curva da média e nas curvas da média mais 2 desvios padrões e média menos 2 desvios padrões.

In [6]:
interactive_plot = interactive(generate_plot_function("linear_regression_data.csv"), 
                               alpha=widgets.IntSlider(min=1, max=100, value=2),
                              beta=widgets.IntSlider(min=1, max=100, value=50))
interactive_plot

interactive(children=(IntSlider(value=2, description='alpha', min=1), IntSlider(value=50, description='beta', …

## Questão 2

Implemente um modelo de regressão polinomial Bayesiana para os dados disponı́veis em *polynomial_regression_data.csv*.

* Utilize um modelo polinomial de grau 5.
* Apresente um gráfico contendo os dados e uma representação da distribuição preditiva encontrada (escolha um método de aproximação).
* Esta representação consistirá na curva da média e nas curvas da média mais 2 desvios padrões e média menos 2 desvios padrões.

In [7]:
interactive_plot = interactive(
    generate_plot_function(
        "polynomial_regression_data.csv", 
        basis_function=polynomial_basis_function,
        figsize=(10,20),
        models_title=False
    ), 
    alpha=widgets.IntSlider(min=1, max=100, value=2),
    beta=widgets.IntSlider(min=1, max=100, value=50)
)
interactive_plot

interactive(children=(IntSlider(value=2, description='alpha', min=1), IntSlider(value=50, description='beta', …